Вот тут мы импортируем библиотеки   pandas   и   matplotlib.pyplot .
 'as' помогает нам избежать больших названий библиотек 
и присвоить им свое имя: pd и plt соответственно

In [136]:
import pandas as pd 
import matplotlib.pyplot as plt

Здесь мы в качестве Х и У берем обучающую выборку. Это набор объктов с известными исходами. В качестве х берут набор объуетов, в качестве у - исходы или ответы.

In [137]:
x = pd.read_csv('train_v2.csv',index_col = 0)
y = pd.read_csv('target.csv',index_col = 0)['radiant_won']

Взглянем на содержимое наших таблиц

In [138]:
x.head()

,lobby_type,r1_hero,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r1_role,r1_avg_tower_damage_per_min,...,dire_solar_crest_count,dire_smoke_of_deceit_count,dire_tome_of_knowledge_count,dire_veil_of_discord_count,dire_rod_of_atos_count,dire_abyssal_blade_count,dire_heavens_halberd_count,dire_ring_of_aquila_count,dire_glimmer_cape_count,teamfight_start
fight_id,,,,,,,,,,,,,,,,,,,,,
0,0,21,22443,17670,165,13,6,30,2,2.557427,...,0,1,2,0,0,0,0,0,0,2205.13658
1,0,22,18303,14005,109,9,10,38,5,0.021600,...,0,4,3,0,1,0,0,0,0,2472.34867
2,7,76,22313,17257,247,4,11,51,2,0.072789,...,0,6,4,1,0,0,0,1,2,2835.45308
3,7,4,15055,11879,112,11,7,27,4,0.207966,...,0,1,1,0,0,0,0,2,0,1938.16395
4,7,43,13003,12716,167,0,10,19,3,0.268209,...,0,3,3,0,0,0,0,0,0,2184.62796


In [139]:
y.head()

fight_id
0     True
1    False
2    False
3    False
4    False
Name: radiant_won, dtype: bool

Нам требуется обучить нашу модель по ответам из таблицы у, чтобы она предсказала победу на других данных. Как происходит предсказание? Модель при обучении ищет зависимости победы или поражения (1 или 0) от набора данных в таблице Х

Теперь возьмем данные из файла с расширением json. В нем хранится характеристика каждого героя. Его способности, сила, интеллект и ловкость.

In [118]:
import json

with open('dictionaries/heroes.json') as f:
          heroes = json.load(f)
          
          

Взглянем на наши данные

In [119]:
print(heroes)

{'1': {'id': 1, 'name': 'npc_dota_hero_antimage', 'localized_name': 'Anti-Mage', 'primary_attr': 'agi', 'attack_type': 'Melee', 'roles': ['Carry', 'Escape', 'Nuker'], 'img': '/apps/dota2/images/heroes/antimage_full.png?', 'icon': '/apps/dota2/images/heroes/antimage_icon.png', 'base_health': 200, 'base_health_regen': 1.5, 'base_mana': 75, 'base_mana_regen': 0.9, 'base_armor': -1, 'base_mr': 25, 'base_attack_min': 27, 'base_attack_max': 31, 'base_str': 22, 'base_agi': 22, 'base_int': 12, 'str_gain': 1.3, 'agi_gain': 2.8, 'int_gain': 1.8, 'attack_range': 150, 'projectile_speed': 0, 'attack_rate': 1.45, 'move_speed': 310, 'turn_rate': 0.5, 'cm_enabled': True, 'legs': 2}, '2': {'id': 2, 'name': 'npc_dota_hero_axe', 'localized_name': 'Axe', 'primary_attr': 'str', 'attack_type': 'Melee', 'roles': ['Initiator', 'Durable', 'Disabler', 'Jungler'], 'img': '/apps/dota2/images/heroes/axe_full.png?', 'icon': '/apps/dota2/images/heroes/axe_icon.png', 'base_health': 200, 'base_health_regen': 4.25, 'ba

Отсюда мы можем брать любые значения. К примеру, посмотрим, сколько изначальной ловкости у 31-го героя в списке. Поскольку нумерация героев в словаре происходит с 1, то 31-й герой будет под номером 31

In [120]:
heroes['31']['base_agi']

15

Теперь, чтобы этот словарь был максимально полезен, я подсчитал уровни героев, исходя из открытых данных в интернете. К примеру, если у героя 199 опыта - он имеет 1-й уровень. Если же 200 опыта, то уже второй. 
Вот функция, которая сама считает уровни. Можно было бы написать ее меньше, но читаемость была бы хуже.

In [121]:
r1_lvl = []
r2_lvl = []
r3_lvl = []
r4_lvl = []
r5_lvl = []
d1_lvl = []
d2_lvl = []
d3_lvl = []
d4_lvl = []
d5_lvl = []
def XpToLvl(xp):
    if (xp < 200):
        return 1
    elif (xp >= 200 and xp < 500):
        return 2
    elif (xp >= 500 and xp < 900):
        return 3
    elif (xp >= 900 and xp < 1400):
        return 4
    elif (xp >= 1400 and xp < 2000):
        return 5
    elif (xp >= 2000 and xp < 2615):
        return 6
    elif (xp >= 2615 and xp < 3425):
        return 7
    elif (xp >= 3425 and xp < 3890):
        return 8
    elif (xp >= 3890 and xp < 4550):
        return 9
    elif (xp >= 4550 and xp < 5225):
        return 10
    elif (xp >= 5225 and xp < 6000):
        return 11
    elif (xp >= 6000 and xp < 7175):
        return 12
    elif (xp >= 7175 and xp < 8375):
        return 13
    elif (xp >= 8375 and xp < 9600):
        return 14
    elif (xp >= 9600 and xp < 10850):
        return 15
    elif (xp >= 10850 and xp < 12125):
        return 16
    elif (xp >= 12125 and xp < 13500):
        return 17
    elif (xp >= 13500 and xp < 14900):
        return 18
    elif (xp >= 14900 and xp < 16325):
        return 19
    elif (xp >= 16325 and xp < 17925):
        return 20
    elif (xp >= 17925 and xp < 19825):
        return 21
    elif (xp >= 19825 and xp < 22025):
        return 22
    elif (xp >= 22025 and xp < 24525):
        return 23
    elif (xp >= 24525 and xp < 27500):
        return 24
    elif (xp >= 27500):
        return 25



Применим нашу функцию для таблицы

In [122]:


r1_gg = x.apply(lambda e: XpToLvl(e['r1_xp']), axis = 1)

r2_gg = x.apply(lambda e: XpToLvl(e['r2_xp']), axis = 1)

r3_gg = x.apply(lambda e: XpToLvl(e['r3_xp']), axis = 1)

r4_gg = x.apply(lambda e: XpToLvl(e['r4_xp']), axis = 1)

r5_gg = x.apply(lambda e: XpToLvl(e['r5_xp']), axis = 1)

d1_gg = x.apply(lambda e: XpToLvl(e['d1_xp']), axis = 1)

d2_gg = x.apply(lambda e: XpToLvl(e['d2_xp']), axis = 1)

d3_gg = x.apply(lambda e: XpToLvl(e['d3_xp']), axis = 1)

d4_gg = x.apply(lambda e: XpToLvl(e['d4_xp']), axis = 1)

d5_gg = x.apply(lambda e: XpToLvl(e['d5_xp']), axis = 1)

columns = ['r1_xp', 'r2_xp', 'r3_xp', 'r4_xp', 'r5_xp', 'd1_xp', 'd2_xp', 'd3_xp', 'd4_xp', 'd5_xp']
#x.drop(columns, axis=1, inplace=True)
x['r1_lvl'] = r1_gg
x['r2_lvl'] = r2_gg
x['r3_lvl'] = r3_gg
x['r4_lvl'] = r4_gg
x['r5_lvl'] = r5_gg
x['d1_lvl'] = d1_gg
x['d2_lvl'] = d2_gg
x['d3_lvl'] = d3_gg
x['d4_lvl'] = d4_gg
x['d5_lvl'] = d5_gg

In [123]:
x.head()

,lobby_type,r1_hero,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r1_role,r1_avg_tower_damage_per_min,...,r1_lvl,r2_lvl,r3_lvl,r4_lvl,r5_lvl,d1_lvl,d2_lvl,d3_lvl,d4_lvl,d5_lvl
fight_id,,,,,,,,,,,,,,,,,,,,,
0,0,21,22443,17670,165,13,6,30,2,2.557427,...,23,23,20,23,21,21,20,17,20,20
1,0,22,18303,14005,109,9,10,38,5,0.021600,...,21,19,21,18,24,17,22,24,24,24
2,7,76,22313,17257,247,4,11,51,2,0.072789,...,23,22,17,24,21,24,24,24,23,20
3,7,4,15055,11879,112,11,7,27,4,0.207966,...,19,20,18,19,14,15,21,19,19,22
4,7,43,13003,12716,167,0,10,19,3,0.268209,...,17,19,21,23,19,20,20,23,18,22


Составим новые признаки, получив уровни героев и очистим таблицу от лишних значений 

In [124]:
x['radiant_lvl'] = x['r1_lvl'] + x['r2_lvl'] + x['r3_lvl'] + x['r4_lvl'] + x['r5_lvl'] 
x['dire_lvl'] = x['d1_lvl'] + x['d2_lvl'] + x['d3_lvl'] + x['d4_lvl'] + x['d5_lvl'] 
x['dire_lvl'] = x['dire_lvl'] /5
x['radiant_lvl'] = x['radiant_lvl'] / 5
x['level_diff'] = x['radiant_lvl'] - x['dire_lvl']

x['radiant_gold'] = x['r1_gold'] + x['r2_gold'] + x['r3_gold'] + x['r4_gold'] + x['r5_gold']
x['dire_gold'] = x['d1_gold'] + x['d2_gold'] + x['d3_gold'] + x['d4_gold'] + x['d5_gold']
x['g_diff'] = x['radiant_gold'] - x['dire_gold'] 

In [125]:
count = [i for i in x.columns if 'xp' in i]
x.drop(count, axis=1, inplace=True)
count = [i for i in x.columns if 'lvl' in i]
x.drop(count, axis=1, inplace=True)
count = [i for i in x.columns if 'gold' in i]
x.drop(count, axis=1, inplace=True)

In [126]:
x.head()

,lobby_type,r1_hero,r1_lh,r1_kills,r1_deaths,r1_items,r1_role,r1_avg_tower_damage_per_min,r1_avg_kda,r2_hero,...,dire_tome_of_knowledge_count,dire_veil_of_discord_count,dire_rod_of_atos_count,dire_abyssal_blade_count,dire_heavens_halberd_count,dire_ring_of_aquila_count,dire_glimmer_cape_count,teamfight_start,level_diff,g_diff
fight_id,,,,,,,,,,,,,,,,,,,,,
0,0,21,165,13,6,30,2,2.557427,5.000000,6,...,2,0,0,0,0,0,0,2205.13658,2.4,20724
1,0,22,109,9,10,38,5,0.021600,2.000000,16,...,3,0,1,0,0,0,0,2472.34867,-1.6,-27187
2,7,76,247,4,11,51,2,0.072789,0.750000,98,...,4,1,0,0,0,1,2,2835.45308,-1.6,-18760
3,7,4,112,11,7,27,4,0.207966,2.636364,44,...,1,0,0,0,0,2,0,1938.16395,-1.2,1596
4,7,43,167,0,10,19,3,0.268209,0.562500,36,...,3,0,0,0,0,0,0,2184.62796,-0.8,622


А теперь к самому созданию модели! 
Разбиваем таблицы на обучающую и валидационную выборки

In [140]:
from sklearn.model_selection import train_test_split

x_train, x_validation, y_train, y_validation = train_test_split(x, y, test_size=.33, random_state=1)

Заполняем пропуски нулями

In [141]:
x_train = x_train.fillna(0)
x_validation = x_validation.fillna(0)

Обучу несколько моделей и проверю их точность.
Первая модель - случайный лес

In [142]:
from sklearn.ensemble import RandomForestClassifier

model_0 = RandomForestClassifier(n_estimators=75, max_depth=4, random_state=322)
model_0.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=75, n_jobs=1,
            oob_score=False, random_state=322, verbose=0, warm_start=False)

Вторая модель - градиентный бустинг

In [143]:
from sklearn.ensemble import GradientBoostingClassifier

model_1 = GradientBoostingClassifier(n_estimators=36, max_depth=5, random_state=322)
model_1.fit(x_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=5,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=36,
              presort='auto', random_state=322, subsample=1.0, verbose=0,
              warm_start=False)

И градиентный бустинг без параметров

In [144]:
from sklearn.ensemble import GradientBoostingClassifier

model_2 = GradientBoostingClassifier()
model_2.fit(x_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

Проверю точность каждой модели

In [145]:
from sklearn.metrics import accuracy_score

print('Train Accuracy:', accuracy_score(y_train, model_0.predict(x_train)))
print('Validation Accuracy:', accuracy_score(y_validation, model_0.predict(x_validation)))

Train Accuracy: 0.6390780831914376
Validation Accuracy: 0.6325925925925926


In [146]:
from sklearn.metrics import accuracy_score

print('Train Accuracy:', accuracy_score(y_train, model_1.predict(x_train)))
print('Validation Accuracy:', accuracy_score(y_validation, model_1.predict(x_validation)))

Train Accuracy: 0.6654402821697883
Validation Accuracy: 0.6326543209876543


In [147]:
from sklearn.metrics import accuracy_score

print('Train Accuracy:', accuracy_score(y_train, model_2.predict(x_train)))
print('Validation Accuracy:', accuracy_score(y_validation, model_2.predict(x_validation)))

Train Accuracy: 0.6558927268304549
Validation Accuracy: 0.6348765432098765
